In [1]:
import scipy.io as sc

In [3]:
data = sc.loadmat('train_data.mat')

In [13]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'label'])

In [256]:
#len(data['data'])

11548

## Task 1

In [73]:
import numpy as np
import pandas as pd
N = len(data['data'])

In [55]:
def find_mean_std(index):
    flatten = (data['data'][index]).flatten()
    mean = np.mean(flatten)
    std = np.std(flatten)
    return mean, std

### Find the mean and Standard Deviation for each of the 11548 images

In [61]:
mean_each_img, std_each_img = [], []
for i in range(N):
    tmp_mean, tmp_std = find_mean_std(i)
    mean_each_img.append(tmp_mean)
    std_each_img.append(tmp_std)

In [62]:
M1 = np.mean(mean_each_img)
S1 = np.std(mean_each_img)

M2 = np.mean(std_each_img)
S2 = np.std(std_each_img)

In [110]:
Y1, Y2 = np.zeros(N), np.zeros(N)
for i in range(N):
    Y1[i]=(mean_each_img[i]-M1)/S1
    Y2[i]=(std_each_img[i]-M2)/S2

#### Y1 and Y2 for each image has been calculated

In [349]:
Y1, M1,S1

(array([ 0.15069489, -0.97386658, -0.60346935, ..., -0.80933654,
        -0.42215888,  0.14114505]),
 32.50435107148159,
 9.349449452973936)

In [258]:
Y2, M2, S2

(array([ 0.12996069, -0.93473171, -0.6292426 , ..., -0.83222958,
        -0.31853427,  0.41329776]),
 76.44042397326842,
 10.509725393032216)

## Task 2

In [80]:
image_3, image_7 = [], []

In [81]:
for i in range(N):
    if data['label'][0][i] == 3:
        image_3.append(i)
    else:
        image_7.append(i)

In [83]:
len(image_3), len(image_7)

(5713, 5835)

### Calculations for Image 3

In [114]:
split_Y1 = [Y1[i] for i in image_3]
split_Y2 = [Y2[i] for i in image_3]

In [115]:
mean_Y1_3, mean_Y2_3 = np.mean(split_Y1), np.mean(split_Y2)
mean_Y1_3, mean_Y2_3

(0.3768799644438678, 0.31851854940980523)

In [117]:
mu_3 = np.array([mean_Y1_3, mean_Y2_3])
mu_3

array([0.37687996, 0.31851855])

In [148]:
discriminant_3 = 0

In [152]:
for i in image_3:
    y_3 = np.array([Y1[i], Y2[i]])
    term_1 = (y_3-mu_3)
    discriminant_3 += term_1 * term_1[None].T
discriminant_3 = discriminant_3/len(image_3)
discriminant_3

array([[1.04928923, 0.98735728],
       [0.98735728, 0.96056405]])

### Calculations for Image 7

In [101]:
split_Y1 = [Y1[i] for i in image_7]
split_Y2 = [Y2[i] for i in image_7]

In [102]:
mean_Y1_7, mean_Y2_7 = np.mean(split_Y1), np.mean(split_Y2)
mean_Y1_7, mean_Y2_7

(-0.3690000405943126, -0.31185886422934433)

In [103]:
std_Y1_7, std_Y2_7 = np.std(split_Y1), np.std(split_Y2)
std_Y1_7, std_Y2_7

(0.8226125232643172, 0.9177161877589698)

In [146]:
mu_7 = np.array([mean_Y1_7, mean_Y2_7])
mu_7

array([-0.36900004, -0.31185886])

In [150]:
discriminant_7 = 0

In [153]:
for i in image_7:
    y_7 = np.array([Y1[i], Y2[i]])
    term_1 = (y_7-mu_7)
    discriminant_7 += term_1 * term_1[None].T
discriminant_7 = discriminant_7/len(image_7)
discriminant_7

array([[0.67680733, 0.74447217],
       [0.74447217, 0.84231953]])

### Density Estimation Calculation

In [154]:
# For digit 3
mu_3, discriminant_3

(array([0.37687996, 0.31851855]),
 array([[1.04928923, 0.98735728],
        [0.98735728, 0.96056405]]))

In [155]:
# For digit 7
mu_7, discriminant_7

(array([-0.36900004, -0.31185886]),
 array([[0.67680733, 0.74447217],
        [0.74447217, 0.84231953]]))

## Task 3

In [303]:
# p(w1/x) = P(w1).p(x|w1)/P(X)
# p(w2/x) = P(w2).p(x|w1)/P(X)
# choose w1 given x iff p(w1|x)>p(w2|x) otherwise choose w2
# so choose w1 when P(w1).P(x|w1)/p(x)>P(w2)p(x|w2)/p(x) otherwise choose w2
post_prob_w1 = np.zeros(N)
post_prob_w2 = np.zeros(N)

### Given P(3) = 0.5 and P(7) = 0.5

In [318]:
from scipy.stats import multivariate_normal
cond_3 = multivariate_normal.pdf(np.array([Y1, Y2]).T, mean=mu_3, cov=discriminant_3)
cond_7 = multivariate_normal.pdf(np.array([Y1, Y2]).T, mean=mu_7, cov=discriminant_7)
prob_x = 0.5 * cond_3 + 0.5 * cond_7
post_prob_w1 = (0.5 * cond_3)/prob_x
post_prob_w2 = (0.5 * cond_7)/prob_x

In [324]:
len(prob_x)

11548

In [320]:
# Error occurs in our system when we choose class w2 for x when p(w1|x)>p(w2|x) and vice versa

In [321]:
P_error_train = 0
for i in range(N):
    P_error_train += min(post_prob_w1[i], post_prob_w2[i])

In [322]:
# Normalizing the error found for the training set
#P_error_train = P_error_train/N
print(f"Error on the training set when P(3)=P(7)=0.5 is {P_error_train/N}")

Error on the training set when P(3)=P(7)=0.5 is 0.26325170639878115


In [323]:
P_error_train

3040.0307054931245

### Given P(3) = 0.3 and P(7) = 0.7

In [276]:
#post_3_s2 = multivariate_normal.pdf(np.array([Y1, Y2]).T, mean=mu_3, cov=discriminant_3)
#post_7_s2 = multivariate_normal.pdf(np.array([Y1, Y2]).T, mean=mu_7, cov=discriminant_7)
prob_x = 0.3 * cond_3 + 0.7 * cond_7
post_prob_w1_s2 = (0.3 * cond_3)/prob_x
post_prob_w2_s2 = (0.7 * cond_7)/prob_x

In [277]:
# Error occurs in our system when we choose class w2 for x when p(w1|x)>p(w2|x) and vice versa

In [278]:
P_error_train_s2 = 0
for i in range(N):
    P_error_train_s2 += min(post_prob_w1_s2[i], post_prob_w2_s2[i])

In [279]:
# Normalizing the error found for the training set
#P_error_train_s2 = P_error_train_s2/N
print(f"Error on the training set when P(3)=0.3 P(7)=0.7 is {P_error_train_s2/N}")

Error on the training set when P(3)=0.3 P(7)=0.7 is 0.17599686224317274


In [280]:
P_error_train_s2

2032.4117651841589

### Performing Task 3 on Test data

In [218]:
data_test = sc.loadmat("test_data.mat")

In [219]:
data_test.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'label'])

In [220]:
data_test['label']

array([[3, 3, 3, ..., 7, 7, 7]])

In [224]:
N_test = len(data_test['data'])

In [225]:
def find_mean_std_test(index):
    flatten = (data_test['data'][index]).flatten()
    mean = np.mean(flatten)
    std = np.std(flatten)
    return mean, std

In [226]:
mean_each_img_test, std_each_img_test = [], []
for i in range(N_test):
    tmp_mean, tmp_std = find_mean_std_test(i)
    mean_each_img_test.append(tmp_mean)
    std_each_img_test.append(tmp_std)

In [331]:
Y1_test, Y2_test = np.zeros(N_test), np.zeros(N_test)
for i in range(N_test):
    Y1_test[i]=(mean_each_img_test[i]-M1)/S1
    Y2_test[i]=(std_each_img_test[i]-M2)/S2

#### Y1 and Y2 for the testing data has now been calculated

In [342]:
Y1_test, Y2_test

(array([ 1.41659394,  0.41809031,  1.40213276, ..., -1.03157488,
        -0.09323522, -1.35831572]),
 array([ 1.29177314,  0.5888807 ,  1.2132882 , ..., -1.07742787,
         0.12317551, -1.53804101]))

In [343]:
post_prob_w1_test_type1 = np.zeros(N_test)
post_prob_w2_test_type1 = np.zeros(N_test)

### Given P(3) = 0.5 and P(7) = 0.5

In [344]:
cond_3_test = multivariate_normal.pdf(np.array([Y1_test, Y2_test]).T, mean=mu_3, cov=discriminant_3)
cond_7_test = multivariate_normal.pdf(np.array([Y1_test, Y2_test]).T, mean=mu_7, cov=discriminant_7)
prob_x_test = 0.5 * cond_3_test + 0.5 * cond_7_test
post_prob_w1_test_type1 = (0.5 * cond_3_test)/prob_x_test
post_prob_w2_test_type1 = (0.5 * cond_7_test)/prob_x_test

In [345]:
P_error_test_s1 = 0
for i in range(N_test):
    P_error_test_s1 += min(post_prob_w1_test_type1[i], post_prob_w2_test_type1[i])

In [346]:
# Normalizing the error found for the test set
print(f"Error on the test set when P(3)=P(7)=0.5 is {P_error_test_s1/N_test}")

Error on the test set when P(3)=P(7)=0.5 is 0.2551125412684881


In [337]:
P_error_test_s1

736.2547941008567

### Given P(3) = 0.3 and P(7) = 0.7

In [338]:
prob_x_test = 0.3 * cond_3_test + 0.7 * cond_7_test
post_prob_w1_test_type2 = (0.3 * cond_3_test)/prob_x_test
post_prob_w2_test_type2 = (0.7 * cond_7_test)/prob_x_test

In [339]:
P_error_test_s2 = 0
for i in range(N_test):
    P_error_test_s2 += min(post_prob_w1_test_type2[i], post_prob_w2_test_type2[i])

In [340]:
# Normalizing the error found for the test set
print(f"Error on the test set when P(3)=0.3 P(7)=0.7 is {P_error_test_s2/N_test}")

Error on the test set when P(3)=0.3 P(7)=0.7 is 0.17138501127954242


In [347]:
P_error_test_s2

494.61714255275945